In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 931.6/931.6 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
dataset=pd.read_csv('manufacturing_6G_dataset.csv')
print(dataset)

                 Timestamp  Machine_ID Operation_Mode  Temperature_C  \
0      2024-01-01 00:00:00          39           Idle      74.137590   
1      2024-01-01 00:01:00          29         Active      84.264558   
2      2024-01-01 00:02:00          15         Active      44.280102   
3      2024-01-01 00:03:00          43         Active      40.568502   
4      2024-01-01 00:04:00           8           Idle      75.063817   
...                    ...         ...            ...            ...   
99995  2024-03-10 10:35:00           3           Idle      71.143556   
99996  2024-03-10 10:36:00          23           Idle      77.256970   
99997  2024-03-10 10:37:00          10         Active      52.800333   
99998  2024-03-10 10:38:00          47         Active      79.967251   
99999  2024-03-10 10:39:00          32         Active      34.318703   

       Vibration_Hz  Power_Consumption_kW  Network_Latency_ms  Packet_Loss_%  \
0          3.500595              8.612162           10.

In [ ]:
df=dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 13 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Timestamp                      100000 non-null  object 
 1   Machine_ID                     100000 non-null  int64  
 2   Operation_Mode                 100000 non-null  object 
 3   Temperature_C                  100000 non-null  float64
 4   Vibration_Hz                   100000 non-null  float64
 5   Power_Consumption_kW           100000 non-null  float64
 6   Network_Latency_ms             100000 non-null  float64
 7   Packet_Loss_%                  100000 non-null  float64
 8   Quality_Control_Defect_Rate_%  100000 non-null  float64
 9   Production_Speed_units_per_hr  100000 non-null  float64
 10  Predictive_Maintenance_Score   100000 non-null  float64
 11  Error_Rate_%                   100000 non-null  float64
 12  Efficiency_Status              

In [ ]:
df.describe()

,Machine_ID,Temperature_C,Vibration_Hz,Power_Consumption_kW,Network_Latency_ms,Packet_Loss_%,Quality_Control_Defect_Rate_%,Production_Speed_units_per_hr,Predictive_Maintenance_Score,Error_Rate_%
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,25.499330,60.041458,2.549959,5.745929,25.555620,2.493418,5.008806,275.916324,0.499385,7.504100
std,14.389439,17.323238,1.414127,2.451271,14.120758,1.443273,2.883666,130.096892,0.288814,4.335896
min,1.000000,30.000138,0.100011,1.500183,1.000025,0.000026,0.000449,50.000375,0.000003,0.000112
25%,13.000000,45.031596,1.323214,3.627318,13.355118,1.245026,2.521591,162.873618,0.248166,3.750148
50%,25.000000,60.033597,2.549441,5.755460,25.536079,2.487667,5.003569,276.648922,0.499209,7.504145
75%,38.000000,74.967217,3.776459,7.860267,37.796372,3.741252,7.506127,388.812761,0.748810,11.273189
max,50.000000,89.998979,4.999974,9.999889,49.999917,4.999975,9.999900,499.996768,0.999978,14.999869


In [ ]:
df['Efficiency_Status'].unique()

array(['Low', 'Medium', 'High'], dtype=object)

In [ ]:
df.head()

,Timestamp,Machine_ID,Operation_Mode,Temperature_C,Vibration_Hz,Power_Consumption_kW,Network_Latency_ms,Packet_Loss_%,Quality_Control_Defect_Rate_%,Production_Speed_units_per_hr,Predictive_Maintenance_Score,Error_Rate_%,Efficiency_Status
0,2024-01-01 00:00:00,39,Idle,74.137590,3.500595,8.612162,10.650542,0.207764,7.751261,477.657391,0.344650,14.965470,Low
1,2024-01-01 00:01:00,29,Active,84.264558,3.355928,2.268559,29.111810,2.228464,4.989172,398.174747,0.769848,7.678270,Low
2,2024-01-01 00:02:00,15,Active,44.280102,2.079766,6.144105,18.357292,1.639416,0.456816,108.074959,0.987086,8.198391,Low
3,2024-01-01 00:03:00,43,Active,40.568502,0.298238,4.067825,29.153629,1.161021,4.582974,329.579410,0.983390,2.740847,Medium
4,2024-01-01 00:04:00,8,Idle,75.063817,0.345810,6.225737,34.029191,4.796520,2.287716,159.113525,0.573117,12.100686,Low


In [ ]:
df=df.drop(columns='Timestamp')

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['Operation_Mode']=le.fit_transform(df['Operation_Mode'])
print(df['Operation_Mode'].unique())

mapping = {'Low': 0, 'Medium': 1, 'High': 2}

# Apply mapping
df['Efficiency_Status'] = df['Efficiency_Status'].map(mapping)
df['Efficiency_Status'].unique()

[1 0 2]


array([0, 1, 2])

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split


# 70% Train, 15% Validation, 15% Test
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42, shuffle=True)
valid_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, shuffle=True)

# Save datasets as CSV
train_df.to_csv('train_industry.csv', index=False)
valid_df.to_csv('valid_industry.csv', index=False)
test_df.to_csv('test_industry.csv', index=False)

print("✅ Dataset successfully split into Train (70%), Validation (15%), and Test (15%)!")


✅ Dataset successfully split into Train (70%), Validation (15%), and Test (15%)!


In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
import pandas as pd


class CSVDataLoader(Dataset):
    def __init__(self, csv_file, input_dim, output_dim):
        self.data = pd.read_csv(csv_file)

        # Drop index if it exists
        self.data.drop(columns=['index'], errors='ignore', inplace=True)

        # Prepare input (features) and output (target) tensors
        self.x = torch.tensor(self.data.iloc[:, :input_dim].values, dtype=torch.float32)  # Use float for inputs


        self.y = torch.tensor(self.data['Efficiency_Status'].values, dtype=torch.float32).unsqueeze(1)  # Make sure y has a 2D shape
        # Reshape x to include sequence length dimension (seq_length=1)
        self.x = self.x.unsqueeze(1)  # Shape: (num_samples, seq_length=1, input_size)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

def get_data_loaders(train_csv_file, val_csv_file, input_dim, output_dim, batch_size):
    # Create train and validation datasets
    train_dataset = CSVDataLoader(train_csv_file, input_dim, output_dim)
    val_dataset = CSVDataLoader(val_csv_file, input_dim, output_dim)

    # Create DataLoader objects
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader

def get_test_loader(test_csv_file, input_dim, output_dim, batch_size):
    # Create test dataset using your CSVDataLoader
    test_dataset = CSVDataLoader(test_csv_file, input_dim, output_dim)

    # Create DataLoader object (no shuffling for testing)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return test_loader


In [ ]:
import torch
import torch.nn as nn

class StackedTransformer(nn.Module):
    def __init__(self, input_size, embed_size, num_heads, num_layers):
        super(StackedTransformer, self).__init__()

        # Embedding layer
        self.embedding = nn.Linear(input_size, embed_size)

        # Stacked Transformer Blocks (Multi-head Attention + Feedforward)
        self.transformer_blocks = nn.ModuleList([
            TransformerBlock(embed_size, num_heads) for _ in range(num_layers)
        ])

        # Final linear layer for binary classification
        self.output_layer = nn.Linear(embed_size, 3)

    def forward(self, x):
        # x shape: (batch_size, seq_length, input_size)
        batch_size, seq_length, _ = x.size()

        # Embedding
        x = self.embedding(x)  # (batch_size, seq_length, embed_size)

        # Permute for multi-head attention (seq_length, batch_size, embed_size)
        x = x.permute(1, 0, 2)

        # Pass through multiple Transformer blocks
        for transformer in self.transformer_blocks:
            x = transformer(x)

        # Revert shape to (batch_size, seq_length, embed_size)
        x = x.permute(1, 0, 2)

        # Pass through output layer
        x = self.output_layer(x)  # (batch_size, seq_length, 1)

        return (x.squeeze(1)) # Squeeze to (batch_size, 1) for binary classification


class TransformerBlock(nn.Module):
    def __init__(self, embed_size, num_heads,dropout=0.1):
        super(TransformerBlock, self).__init__()

        # Multi-head attention
        self.multihead_attention = nn.MultiheadAttention(embed_dim=embed_size, num_heads=num_heads)

        # Feedforward network
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, embed_size),
            nn.ReLU(),
            nn.Linear(embed_size, embed_size)
        )

        # Layer normalization
        self.layer_norm1 = nn.LayerNorm(embed_size)
        self.layer_norm2 = nn.LayerNorm(embed_size)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x):
        # Multi-head attention
        attn_output, _ = self.multihead_attention(x, x, x)

        # Skip connection and layer normalization
        #x = self.layer_norm1(attn_output + x)
        x = self.layer_norm1(x + self.dropout(attn_output))


        # Feedforward
        ff_output = self.feed_forward(x)

        # Skip connection and layer normalization
        x = self.layer_norm2(ff_output + x)

        return x


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchmetrics import Accuracy

best_model_path = "best_model.pth"


def evaluate_model(model, data_loader, criterion, device, epoch, best_val_loss):
    model.eval()
    val_loss = 0.0

    # Initialize Accuracy metric for multi-class
    accuracy = Accuracy(task="multiclass", num_classes=3).to(device)

    with torch.no_grad():
        for x, y in data_loader:
            x, y = x.to(device), y.to(device)

            # Forward pass
            predictions = model(x)
            y = y.squeeze(1).long()

            # Compute loss
            loss = criterion(predictions, y)
            val_loss += loss.item()

            # Update accuracy
            accuracy.update(predictions, y)

    val_loss = val_loss / len(data_loader)

    # Save the model if the validation loss improves
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), best_model_path)
        print(f"Model saved at epoch {epoch + 1} with val_loss: {val_loss:.4f}")

    print(f"Epoch {epoch + 1} Accuracy: {accuracy.compute():.4f}")
    return val_loss, best_val_loss


def train_model(model, train_loader, val_loader, epochs, learning_rate, device):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()  # For multi-class classification
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    best_val_loss = float('inf')  # Initialize with a large value

    for epoch in range(epochs):
        model.train()
        train_loss = 0

        for batch_idx, (x, y) in enumerate(train_loader):
            x, y = x.to(device), y.to(device)

            # Forward pass
            predictions = model(x)
            y = y.squeeze(1).long()
            loss = criterion(predictions, y)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        # Calculate validation loss and update best model
        val_loss, best_val_loss = evaluate_model(model, val_loader, criterion, device, epoch, best_val_loss)

        print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss / len(train_loader):.4f}, Validation Loss: {val_loss:.4f}")

    print("Training completed. Best model saved to:", best_model_path)


# Example usage
if __name__ == "__main__":
    # Define parameters
    input_dim = 11  # Number of input features
    output_dim = 3  # Number of output targets (for multiclass classification)
    embed_size = 128  # Size of the embedding
    num_heads = 16  # Number of attention heads
    batch_size = 256  # Batch size
    learning_rate = 0.0005  # Learning rate
    epochs = 30  # Number of epochs
    num = 8  # Number of transformer layers
    train_csv_file = 'train_industry.csv'  # Training data CSV file
    val_csv_file = 'valid_industry.csv'  # Validation data CSV file
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available

    # Initialize Accuracy metric
    accuracy = Accuracy(task="multiclass", num_classes=3).to(device)

    # Initialize DataLoaders for training and validation datasets
    train_loader, val_loader = get_data_loaders(train_csv_file, val_csv_file, input_dim, output_dim, batch_size)

    # Inspect shapes of one batch to ensure everything is working correctly
    for x, y in train_loader:
        print("Training Batch - X Shape:", x.shape, "Y Shape:", y.shape)
        break

    # Initialize StackedTransformer Model
    model = StackedTransformer(input_size=input_dim, embed_size=embed_size, num_heads=num_heads, num_layers=num)

    # Train the model with validation
    train_model(model, train_loader, val_loader, epochs, learning_rate, device)



Training Batch - X Shape: torch.Size([256, 1, 11]) Y Shape: torch.Size([256, 1])
Model saved at epoch 1 with val_loss: 0.1787
Epoch 1 Accuracy: 0.9218
Epoch 1/30, Train Loss: 0.3593, Validation Loss: 0.1787
Model saved at epoch 2 with val_loss: 0.1478
Epoch 2 Accuracy: 0.9288
Epoch 2/30, Train Loss: 0.1828, Validation Loss: 0.1478
Epoch 3 Accuracy: 0.9265
Epoch 3/30, Train Loss: 0.1621, Validation Loss: 0.1681
Epoch 4 Accuracy: 0.9178
Epoch 4/30, Train Loss: 0.1485, Validation Loss: 0.2126
Model saved at epoch 5 with val_loss: 0.1344
Epoch 5 Accuracy: 0.9431
Epoch 5/30, Train Loss: 0.1439, Validation Loss: 0.1344
Epoch 6 Accuracy: 0.9319
Epoch 6/30, Train Loss: 0.1412, Validation Loss: 0.1385
Epoch 7 Accuracy: 0.9394
Epoch 7/30, Train Loss: 0.1349, Validation Loss: 0.1358
Epoch 8 Accuracy: 0.9412
Epoch 8/30, Train Loss: 0.1330, Validation Loss: 0.1407
Model saved at epoch 9 with val_loss: 0.1252
Epoch 9 Accuracy: 0.9419
Epoch 9/30, Train Loss: 0.1295, Validation Loss: 0.1252
Model save

In [ ]:
import torch
import torch.nn as nn
from torchmetrics import Accuracy, Precision, Recall, F1Score

# Define test function
def test_model(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0.0

    # Initialize metrics for multi-class classification
    accuracy = Accuracy(task="multiclass", num_classes=3).to(device)
    precision = Precision(task="multiclass", num_classes=3, average='macro').to(device)  # Macro for averaging across classes
    recall = Recall(task="multiclass", num_classes=3, average='macro').to(device)
    f1 = F1Score(task="multiclass", num_classes=3, average='macro').to(device)

    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)

            # Forward pass
            predictions = model(x)
            y = y.squeeze(1).long()

            # Compute loss
            loss = criterion(predictions, y)
            test_loss += loss.item()

            # Update metrics
            accuracy.update(predictions, y)
            precision.update(predictions, y)
            recall.update(predictions, y)
            f1.update(predictions, y)

    # Compute final values
    test_loss = test_loss / len(test_loader)
    test_acc = accuracy.compute().item()
    test_precision = precision.compute().item()
    test_recall = recall.compute().item()
    test_f1 = f1.compute().item()

    print(f"Test Loss: {test_loss:.4f}")
    print(f"Accuracy: {test_acc:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1 Score: {test_f1:.4f}")

    return test_loss, test_acc, test_precision, test_recall, test_f1

# Load the best trained model
best_model_path = "best_model.pth"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define model architecture (ensure this matches training)
model = StackedTransformer(input_size=11, embed_size=128, num_heads=16, num_layers=8).to(device)

# Load trained weights
model.load_state_dict(torch.load(best_model_path))
model.to(device)

# Load test dataset
test_csv_file = 'test_industry.csv'
test_loader = get_test_loader(test_csv_file, input_dim=11, output_dim=3, batch_size=256)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Evaluate on the test set
test_loss, test_accuracy, test_precision, test_recall, test_f1 = test_model(model, test_loader, criterion, device)


<ipython-input-54-876c5e41845a>:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_path))


Test Loss: 0.0628
Accuracy: 0.9757, Precision: 0.9531, Recall: 0.8853, F1 Score: 0.9132
